<a href="https://colab.research.google.com/github/jaynariya605/Non-linear-regression/blob/master/assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import the pandas library to read our dataset
import pandas as pd

# Get the train/test split package from sklearn for preparing our dataset to
# train and test the model with
from sklearn.model_selection import train_test_split

# Import the numpy library to work with and manipulate the data
import numpy as np

In [0]:
# import data from google drive
#data = pd.read_csv('train.tsv', sep='\t')
data = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')

In [3]:
# Check the head of the dataframe
data.head(20)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [4]:
# check the shape of df
data.shape

(156060, 4)

In [0]:
# Get number of unique sentences
numSentences = data['SentenceId'].max()

In [0]:
# extract full sentences only from the dataset
fullSentences = []
curSentence = 0
for i in range(data.shape[0]):
  if data['SentenceId'][i]> curSentence:
    fullSentences.append((data['Phrase'][i], data['Sentiment'][i]))
    curSentence = curSentence +1

In [7]:
len(fullSentences)

8544

In [0]:
# put data into a df
all_data = pd.DataFrame(fullSentences,
                                columns=['Phrase', 'Sentiment'])

In [9]:
all_data

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,"This quiet , introspective and entertaining in...",4
2,"Even fans of Ismail Merchant 's work , I suspe...",1
3,A positively thrilling combination of ethnogra...,3
4,Aggressive self-glorification and a manipulati...,1
...,...,...
8539,... either you 're willing to go with this cla...,2
8540,"Despite these annoyances , the capable Claybur...",2
8541,-LRB- Tries -RRB- to parody a genre that 's al...,1
8542,The movie 's downfall is to substitute plot fo...,1


In [10]:
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
documents = []
#convert data into format for the previous labs

#use full dataset
#for i in range(data.shape[0]):
#  tmpWords = word_tokenize(data['Phrase'][i])
#  documents.append((tmpWords, data['Sentiment'][i]))

# Use only complete sentences
for i in range(all_data.shape[0]):
  tmpWords = word_tokenize(all_data['Phrase'][i])
  documents.append((tmpWords, all_data['Sentiment'][i]))

In [12]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations="?:!.,;'\"-()"

#parameters to adjust to see the impact on outcome
remove_stopwords = False
useStemming = False
useLemma = False
removePuncs = True

for l in range(len(documents)):
  label = documents[l][1]
  tmpReview = []
  for w in documents[l][0]:
    newWord = w
    if remove_stopwords and (w in stopwords_en):
      continue
    if removePuncs and (w in punctuations):
      continue
    if useStemming:
      #newWord = porter.stem(newWord)
      newWord = lancaster.stem(newWord)
    if useLemma:
      newWord = wordnet_lemmatizer.lemmatize(newWord)
    tmpReview.append(newWord)
  documents[l] = (' '.join(tmpReview), label)
print(documents[1])

('This quiet introspective and entertaining independent is worth seeking', 4)


In [0]:
all_data = pd.DataFrame(documents,
                                columns=['text', 'sentiment'])

In [0]:
X = all_data['text']

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from keras.utils import to_categorical
# Transform each text into a vector of word counts
vec = CountVectorizer(stop_words="english",
                             ngram_range=(1, 1))
X = vec.fit_transform(X).A
Y = to_categorical(all_data['sentiment'].values)

Using TensorFlow backend.


In [0]:

# Splits the dataset so 70% is used for training and 30% for testing
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.3,random_state = 2003)

In [17]:
x_train.shape

(5980, 14879)

In [18]:
x_test.shape

(2564, 14879)

In [19]:
y_test.shape

(2564, 5)

In [0]:
import numpy as np


In [0]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D , AveragePooling1D
from keras.models import Sequential

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
nrows, ncols = x_train.shape
x_train1 = x_train.reshape(nrows, ncols,1)

nrows, ncols = x_test.shape
x_test1 = x_test.reshape(nrows, ncols,1)



In [25]:
x_train1.shape[1]

14879

In [26]:
model2 = Sequential()
model2.add(Conv1D(32, kernel_size=2,input_shape=(x_train1.shape[1],1),padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Conv1D(64, kernel_size=2, padding='same', activation='relu'))
model2.add(AveragePooling1D(pool_size=2))

model2.add(Flatten())

# Output layer
model2.add(Dense(5, activation='softmax'))
model2.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 14879, 32)         96        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7439, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 7439, 64)          4160      
_________________________________________________________________
average_pooling1d_1 (Average (None, 3719, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 238016)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 1190085   
Total params: 1,194,341
Trainable params: 1,194,341
Non-trainable params: 0
__________________________________________

In [27]:
model2.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model2.fit(x_train1, y_train, validation_data=(x_test1, y_test), epochs=30, batch_size=32, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 5980 samples, validate on 2564 samples
Epoch 1/30





5980/5980 [==============================] - 21s 4ms/step - loss: 1.5117 - acc: 0.3431 - f1_m: 0.0556 - precision_m: 0.3523 - recall_m: 0.0316 - val_loss: 1.4196 - val_acc: 0.3849 - val_f1_m: 0.1226 - val_precision_m: 0.5212 - val_recall_m: 0.0710
Epoch 2/30
5980/5980 [==============================] - 9s 2ms/step - loss: 0.9692 - acc: 0.6164 - f1_m: 0.5545 - precision_m: 0.7249 - recall_m: 0.4555 - val_loss: 1.5920 - val_acc: 0.3729 - val_f1_m: 0.2729 - val_precision_m: 0.4090 - val_recall_m: 0.2067
Epoch 3/30
5980/5980 [==============================] - 9s 2ms/step - loss: 0.5188 - acc: 0.8038 - f1_m: 0.7950 - precision_m: 0.8525 - recall_m: 0.7465 - val_loss: 2.2990 - val_acc: 0.3584 - val_f1_m: 0.3363 - val_precision_m: 0.3775 - val_recall_m: 0.3042
Epoch 4/30
5980/5980 [==============================] - 9s 2ms/step - loss:

In [0]:
model2.save('model2.h5')

In [0]:
model3 = Sequential()
model3.add(Conv1D(32, kernel_size=3,input_shape=(x_train1.shape[1],1), padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))

model3.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))

model3.add(Flatten())

# Output layer
model3.add(Dense(5, activation='softmax'))

In [30]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model3.fit(x_train1, y_train, validation_data=(x_test1, y_test), epochs=10, batch_size=30, verbose=1)

Train on 5980 samples, validate on 2564 samples
Epoch 1/10
5980/5980 [==============================] - 10s 2ms/step - loss: 1.5030 - acc: 0.3410 - f1_m: 0.0475 - precision_m: 0.3054 - recall_m: 0.0266 - val_loss: 1.4279 - val_acc: 0.3814 - val_f1_m: 0.0859 - val_precision_m: 0.4431 - val_recall_m: 0.0484
Epoch 2/10
5980/5980 [==============================] - 9s 2ms/step - loss: 0.9292 - acc: 0.6306 - f1_m: 0.5768 - precision_m: 0.7389 - recall_m: 0.4823 - val_loss: 1.6178 - val_acc: 0.3662 - val_f1_m: 0.2834 - val_precision_m: 0.4111 - val_recall_m: 0.2180
Epoch 3/10
5980/5980 [==============================] - 9s 2ms/step - loss: 0.4814 - acc: 0.8221 - f1_m: 0.8153 - precision_m: 0.8722 - recall_m: 0.7676 - val_loss: 2.3963 - val_acc: 0.3506 - val_f1_m: 0.3272 - val_precision_m: 0.3659 - val_recall_m: 0.2968
Epoch 4/10
5980/5980 [==============================] - 9s 2ms/step - loss: 0.2490 - acc: 0.9159 - f1_m: 0.9138 - precision_m: 0.9369 - recall_m: 0.8926 - val_loss: 3.3974 - val

In [0]:
model3.save_weights('model3.h5')

In [32]:
from keras import optimizers
lr = 1e-3
Nadam=optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()
model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy',f1_m,precision_m, recall_m])



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                476160    
_________________________________________________________________
dense_8 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 45        
Total params: 476,869
Trainable params: 476,869
Non-trainable params: 0
_________________________________________________________________


In [33]:
history = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15, batch_size=10000, verbose=1)

Train on 5980 samples, validate on 2564 samples
Epoch 1/15
5980/5980 [==============================] - 1s 193us/step - loss: 1.6068 - acc: 0.2610 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.6032 - val_acc: 0.2886 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/15
5980/5980 [==============================] - 1s 100us/step - loss: 1.5948 - acc: 0.3764 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.5992 - val_acc: 0.3085 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/15
5980/5980 [==============================] - 1s 85us/step - loss: 1.5831 - acc: 0.4258 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.5947 - val_acc: 0.3233 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/15
5980/5980 [==============================] - 1s 85us/step - loss: 1.5699 - acc: 0.4552 - f

In [34]:
model.evaluate(x_test, y_test, batch_size=10000, verbose=1)

2564/2564 [==============================] - 0s 59us/step


[1.5182139873504639,
 0.36466458439826965,
 0.0015570255927741528,
 0.4000000059604645,
 0.0007800312014296651]